#### Daily Stock Movement and RSI Analysis

How to start with Python? Make sure to run the following code.

In [13]:
#### mkdir location
#### cd location
#### code .
#### python -m venv venv1
#### venv1/Scripts/activate
#### python -m pip install --upgrade pip
#### pip install -r requirements.txt


In [14]:
# Import Python Libraries
import yfinance as yf
import pandas as pd
import openpyxl as xl
import math
from datetime import datetime, timedelta

### 1. Reading Excel to Extract Stock Quotes
Remember to change the path to the local file location

In [15]:
### Set path
file_path = "dim_stock.xlsx"

### Read dim_stock Excel tab
dim_stock = pd.read_excel(file_path, sheet_name="dim_stock")

#### 1.1. Analyzing Excel Stock Table 
Looking into the dim_stock table:

In [16]:
### Print the first few rows
print(dim_stock.head())

   No     Code   Symbol                   Company Name Category   \
0   1  0037.KL      RGB          RGB INTERNATIONAL BHD    NON CI   
1   3  3182.KL  GENTING                    GENTING BHD        CI   
2   4  3891.KL   MUIIND  MALAYAN UNITED INDUSTRIES BHD    NON CI   
3   5  4219.KL   BJLAND               BERJAYA LAND BHD    NON CI   
4   6  4588.KL      UMW               UMW HOLDINGS BHD    NON CI   

                       Sector  Trading Board  
0  CONSUMER PRODUCTS/ SERVICES          MAIN  
1  CONSUMER PRODUCTS/ SERVICES          MAIN  
2  CONSUMER PRODUCTS/ SERVICES          MAIN  
3  CONSUMER PRODUCTS/ SERVICES          MAIN  
4  CONSUMER PRODUCTS/ SERVICES          MAIN  


In [17]:
### Print the structure of the dim_stock table
print("There are " + str(dim_stock.shape[0]) + " stocks in the table.")
print("###########################")
print(dim_stock.info())

There are 982 stocks in the table.
###########################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   No             982 non-null    int64 
 1   Code           982 non-null    object
 2   Symbol         982 non-null    object
 3   Company Name   982 non-null    object
 4   Category       982 non-null    object
 5   Sector         982 non-null    object
 6   Trading Board  982 non-null    object
dtypes: int64(1), object(6)
memory usage: 53.8+ KB
None


In [18]:
### Drop the duplicated row
dim_stock = dim_stock.drop_duplicates(subset=["Code"])

print("There are " + str(dim_stock.shape[0]) + " unique stocks in the table.")
print("###########################")
print(dim_stock.info())

There are 975 unique stocks in the table.
###########################
<class 'pandas.core.frame.DataFrame'>
Index: 975 entries, 0 to 981
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   No             975 non-null    int64 
 1   Code           975 non-null    object
 2   Symbol         975 non-null    object
 3   Company Name   975 non-null    object
 4   Category       975 non-null    object
 5   Sector         975 non-null    object
 6   Trading Board  975 non-null    object
dtypes: int64(1), object(6)
memory usage: 60.9+ KB
None


#### 1.2. Check Stock Avaibility on Yahoo (can be skipped)
This section is to check the availability of all stocks provided in the Excel on Yahoo Finance. 

In [19]:
# test_start = datetime.today().date() - timedelta(5)

In [20]:
# def check_stock_available(x):   
#     try:   
#         stock = yf.download(x, start=test_start)
#         if len(stock)==0:
#             return False
#         else:
#             return True
#     except:
#         return False

In [21]:
#  ### Run the defined function on the stocks and create a new column
#  dim_stock["Available"] = dim_stock["Code"].apply(lambda x: check_stock_available(x))
#  
#  ### Print the output
#  print(dim_stock.head())

This section is to take the final set of stocks that are only available on Yahoo Finance.

In [22]:
# ### Create a filtered stock table where stock availability = True
# final_stocks = dim_stock[dim_stock.Available == True]
# 
# ### Create a filtered stock table where the stocks cannot be found on Yahoo
# unavailable_stocks = dim_stock[dim_stock.Available == False]
# 
# ### Count the number of stocks that are available & unavailable
# print("There are " + str(final_stocks.shape[0]) + " available stock(s) and " + str(unavailable_stocks.shape[0]) + " unavailable stock(s) on Yahoo.")
# print("###########################")
# 
# ### Print the output
# print(final_stocks.head())

### 2. Defining Variables
Define the start_date and end_date of your analysis.
end_date will automatically be today's date.

In [44]:
### Define date period
start_date = '2023-12-01'
end_date = datetime.today().date().strftime('%Y-%m-%d') #auto-detection of today's date

### print your date range
print("You want to extract the stock data from " + start_date + " to " + end_date + ".")

You want to extract the stock data from 2023-12-01 to 2024-01-29.


#### 3. Choosing 1 Stock as Sample to Run

##### 3.1 Get the Stock Historical Data

In [72]:
### Get the sample stock historical data
stock1 = "7277.KL" #input your sample stock here
stock_history1 = yf.download(stock1, start=start_date, end=end_date)
stock_history1 = stock_history1.reset_index()
print(stock_history1.head(3))

[*********************100%%**********************]  1 of 1 completed
        Date  Open  High   Low  Close  Adj Close   Volume
0 2023-12-01  2.13  2.13  2.06   2.07       2.07  7344600
1 2023-12-04  2.06  2.08  2.04   2.07       2.07  4727400
2 2023-12-05  2.07  2.07  2.03   2.04       2.04  3666700


In [73]:
# Check for NaN or Null value
print(stock_history1[stock_history1.isna().any(axis=1)])

na_count = stock_history1.isna().any(axis=1).sum()

print(f"Number of inactive trading days is {na_count}.")

# Remove rows with NaN or Null
stock_history1 = stock_history1.dropna()

Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume]
Index: []
Number of inactive trading days is 0.


##### 3.2. Calculate Change, Gain, Loss
This section is to calculate base calculation such as Daily Price Change, Gain, and Loss.

In [74]:
### Calculate daily price change
stock_history1["Change"] = stock_history1["Adj Close"].diff()

# Roundup Change to 6 decimal places
stock_history1["Change"] = stock_history1["Change"].apply(lambda x: x if pd.isna(x) else math.ceil(x * 1e6)/1e6)

### Calculate the Gain with Logic
stock_history1["Gain"] = stock_history1["Change"].apply(lambda x: x if x > 0 else 0)

### Calculate the Loss with Logic
stock_history1["Loss"] = stock_history1["Change"].apply(lambda x: abs(x) if x < 0 else 0)

This section is to calculate the rolling average (14 days) of Gain and Loss.

In [75]:
### Calculate the rolling average of Gain
stock_history1["SMA Gain"] = stock_history1["Gain"].rolling(window=14).mean()
stock_history1["SMA Gain"] = stock_history1["SMA Gain"].shift(1)
stock_history1["Prev SMA Gain"] = stock_history1["SMA Gain"].shift(1)

### Calculate the rolling average of Loss
stock_history1["SMA Loss"] = stock_history1["Loss"].rolling(window=14).mean()
stock_history1["SMA Loss"] = stock_history1["SMA Loss"].shift(1)
stock_history1["Prev SMA Loss"] = stock_history1["SMA Loss"].shift(1)

# Rounddown to 6 decimal places
columns = ["SMA Gain", "SMA Loss"]
stock_history1[columns] = stock_history1[columns].map(lambda x: x if pd.isna(x) else math.floor(x * 1e6)/1e6)

In [76]:
print(stock_history1.head(3))

        Date  Open  High   Low  Close  Adj Close   Volume    Change  Gain  \
0 2023-12-01  2.13  2.13  2.06   2.07       2.07  7344600       NaN   0.0   
1 2023-12-04  2.06  2.08  2.04   2.07       2.07  4727400  0.000000   0.0   
2 2023-12-05  2.07  2.07  2.03   2.04       2.04  3666700 -0.029999   0.0   

       Loss  SMA Gain  Prev SMA Gain  SMA Loss  Prev SMA Loss  
0  0.000000       NaN            NaN       NaN            NaN  
1  0.000000       NaN            NaN       NaN            NaN  
2  0.029999       NaN            NaN       NaN            NaN  


This section is to calculate the Average Gain and Loss using the special formula:
1. For the first average, take the simple moving average
2. For the average afterwards, is to take (previous average * 13 + current value)/14

In [77]:
### Special Formula for AVG Gain
stock_history1["AVG Gain"] = 0

for index, row in stock_history1.iterrows():
    # For the first available rolling average, use back its value
    if pd.isna(row["Prev SMA Gain"]):
        stock_history1.at[index, "AVG Gain"] = row["SMA Gain"]
    # Use the result of the previous row as an input for the current row
    else:
        previous_result = stock_history1.at[row.name-1, "AVG Gain"]
        stock_history1.at[index, "AVG Gain"] = math.floor((previous_result*13 + row["Gain"])/14 * 1e6 )/1e6
        
 ### Special Formula for AVG Loss
stock_history1["AVG Loss"] = 0

for index, row in stock_history1.iterrows():
    # For the first available rolling average, use back its value
    if pd.isna(row["Prev SMA Loss"]):
        stock_history1.at[index, "AVG Loss"] = row["SMA Loss"]
    # Use the result of the previous row as an input for the current row
    else:
        previous_result = stock_history1.at[row.name-1, "AVG Loss"]
        stock_history1.at[index, "AVG Loss"] = math.floor((previous_result*13 + row["Loss"])/14 * 1e6 )/1e6

This section is to clean the data to round down the AVG Gain and AVG Loss

In [78]:
# Drop the unused columns
keywords = ["SMA", "Prev"]
stock_history1 = stock_history1.filter(regex=f'^(?!.*{"|".join(keywords)}).*$', axis=1)

# Rounddown to 6 decimal places
#columns = ["AVG Gain", "AVG Loss"]
#stock_history1[columns] = stock_history1[columns].map(lambda x: x if pd.isna(x) else math.floor(x * 1e6)/1e6)

This section is to calculate the Gain/Loss ratio and RSI

In [79]:
### Calculate Gain/Loss ratio with 6 decimal places rounded down
stock_history1["Gain/Loss"] = stock_history1["AVG Gain"]/stock_history1["AVG Loss"]
#stock_history1["Gain/Loss"] = stock_history1["Gain/Loss"].fillna(0)
stock_history1["Gain/Loss"] = stock_history1["Gain/Loss"].apply(lambda x: x if pd.isna(x) else math.floor(x * 1e6)/1e6)



In [80]:
### Calculate RSI with 4 decimal places rounded down
stock_history1["RSI"] = stock_history1["Gain/Loss"].apply(lambda x: 100 if x==0 else 100-(100/(1+x)))
stock_history1["RSI"] = stock_history1["RSI"].apply(lambda x: x if pd.isna(x) else math.floor(x * 1e4)/1e4)

In [81]:
### Extract data starting from 2024 Jan 01, all historical is not run and stored in historical_output
stock_history1 = stock_history1[stock_history1["Date"]>='2024-01-01'].reset_index(drop=True)
display(stock_history1.head(30))

,Date,Open,High,Low,Close,Adj Close,Volume,Change,Gain,Loss,AVG Gain,AVG Loss,Gain/Loss,RSI
0,2024-01-02,2.08,2.09,2.05,2.06,2.06,4216400,-0.009999,0.000000,0.009999,0.010976,0.012163,0.902408,47.4350
1,2024-01-03,2.05,2.06,2.02,2.03,2.03,2915200,-0.029999,0.000000,0.029999,0.010191,0.013437,0.758428,43.1310
2,2024-01-04,2.03,2.07,2.03,2.07,2.07,6280100,0.040000,0.040000,0.000000,0.012320,0.012477,0.987416,49.6834
3,2024-01-05,2.06,2.09,2.05,2.08,2.08,4519600,0.010000,0.010000,0.000000,0.012154,0.011585,1.049115,51.1984
4,2024-01-08,2.08,2.10,2.07,2.08,2.08,8755000,0.000000,0.000000,0.000000,0.011285,0.010757,1.049084,51.1977
5,2024-01-09,2.07,2.08,2.06,2.07,2.07,6147500,-0.009999,0.000000,0.009999,0.010478,0.010702,0.979069,49.4711
6,2024-01-10,2.07,2.07,2.04,2.07,2.07,9891400,0.000000,0.000000,0.000000,0.009729,0.009937,0.979068,49.4711
7,2024-01-11,2.06,2.06,1.99,2.01,2.01,7168100,-0.059999,0.000000,0.059999,0.009034,0.013512,0.668590,40.0691
8,2024-01-12,2.02,2.05,2.01,2.05,2.05,9055700,0.040000,0.040000,0.000000,0.011245,0.012546,0.896301,47.2657
9,2024-01-15,2.05,2.05,1.97,1.98,1.98,10595300,-0.069999,0.000000,0.069999,0.010441,0.016649,0.627124,38.5418


### 4. Create a Main Function
This chapter is to build a function that contains all the logics and steps created in chapter 3 using a sample stock. The function is to be run on all available stocks.  

In [82]:
#### Define a function
def stock_history_function(stock_i):
    stock_history_i = yf.download(stock_i, start=start_date, end=end_date)
    stock_history_i = stock_history_i.reset_index()
    ### Calculate daily price change
    stock_history_i["Change"] = stock_history_i["Adj Close"].diff()
    # Roundup Change to 6 decimal places
    stock_history_i["Change"] = stock_history_i["Change"].apply(lambda x: x if pd.isna(x) else math.ceil(x * 1e6)/1e6)
    ### Calculate the Gain with Logic
    stock_history_i["Gain"] = stock_history_i["Change"].apply(lambda x: x if x > 0 else 0)
    ### Calculate the Loss with Logic
    stock_history_i["Loss"] = stock_history_i["Change"].apply(lambda x: abs(x) if x < 0 else 0)
    
    ### Calculate the rolling average of Gain
    stock_history_i["SMA Gain"] = stock_history_i["Gain"].rolling(window=14).mean()
    stock_history_i["SMA Gain"] = stock_history_i["SMA Gain"].shift(1)
    stock_history_i["Prev SMA Gain"] = stock_history_i["SMA Gain"].shift(1)

    ### Calculate the rolling average of Loss
    stock_history_i["SMA Loss"] = stock_history_i["Loss"].rolling(window=14).mean()
    stock_history_i["SMA Loss"] = stock_history_i["SMA Loss"].shift(1)
    stock_history_i["Prev SMA Loss"] = stock_history_i["SMA Loss"].shift(1)

    # Rounddown to 6 decimal places
    columns_1 = ["SMA Gain", "SMA Loss"]
    stock_history_i[columns] = stock_history_i[columns_1].map(lambda x: x if pd.isna(x) else math.floor(x * 1e6)/1e6)
    
    ### Special Formula for AVG Gain
    stock_history_i["AVG Gain"] = 0

    for index, row in stock_history_i.iterrows():
        # For the first available rolling average, use back its value
        if pd.isna(row["Prev SMA Gain"]):
            stock_history_i.at[index, "AVG Gain"] = row["SMA Gain"]
        # Use the result of the previous row as an input for the current row
        else:
            previous_result = stock_history_i.at[row.name-1, "AVG Gain"]
            stock_history_i.at[index, "AVG Gain"] = math.floor((previous_result*13 + row["Gain"])/14 * 1e6 )/1e6
            
    ### Special Formula for AVG Loss
    stock_history_i["AVG Loss"] = 0

    for index, row in stock_history_i.iterrows():
        # For the first available rolling average, use back its value
        if pd.isna(row["Prev SMA Loss"]):
            stock_history_i.at[index, "AVG Loss"] = row["SMA Loss"]
        # Use the result of the previous row as an input for the current row
        else:
            previous_result = stock_history_i.at[row.name-1, "AVG Loss"]
            stock_history_i.at[index, "AVG Loss"] = math.floor((previous_result*13 + row["Loss"])/14 * 1e6 )/1e6
    
    # Drop the unused columns
    keywords = ["SMA", "Prev"]
    stock_history_i = stock_history_i.filter(regex=f'^(?!.*{"|".join(keywords)}).*$', axis=1)
    
    ### Calculate Gain/Loss ratio with 6 decimal places rounded down
    stock_history_i["Gain/Loss"] = stock_history_i["AVG Gain"]/stock_history_i["AVG Loss"]
    #stock_history1["Gain/Loss"] = stock_history1["Gain/Loss"].fillna(0)
    stock_history_i["Gain/Loss"] = stock_history_i["Gain/Loss"].apply(lambda x: x if pd.isna(x) else math.floor(x * 1e6)/1e6)
    
    ### Calculate RSI with 4 decimal places rounded down
    stock_history_i["RSI"] = stock_history_i["Gain/Loss"].apply(lambda x: 100 if x==0 else 100-(100/(1+x)))
    stock_history_i["RSI"] = stock_history_i["RSI"].apply(lambda x: x if pd.isna(x) else math.floor(x * 1e4)/1e4)

    ### Map to the stock code
    stock_history_i["Code"] = stock_i
    
    ### Reset the index and extract data from 2024 Jan 01 onwards
    stock_history_i = stock_history_i[stock_history_i["Date"]>='2024-01-01'].reset_index(drop=True)
    
    ### Return the dataframe
    return stock_history_i
    

In [83]:
# Test for 1 Stock
stock_history_function("7277.KL")

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,Change,Gain,Loss,AVG Gain,AVG Loss,Gain/Loss,RSI,Code
0,2024-01-02,2.08,2.09,2.05,2.06,2.06,4216400,-0.009999,0.000000,0.009999,0.010976,0.012163,0.902408,47.4350,7277.KL
1,2024-01-03,2.05,2.06,2.02,2.03,2.03,2915200,-0.029999,0.000000,0.029999,0.010191,0.013437,0.758428,43.1310,7277.KL
2,2024-01-04,2.03,2.07,2.03,2.07,2.07,6280100,0.040000,0.040000,0.000000,0.012320,0.012477,0.987416,49.6834,7277.KL
3,2024-01-05,2.06,2.09,2.05,2.08,2.08,4519600,0.010000,0.010000,0.000000,0.012154,0.011585,1.049115,51.1984,7277.KL
4,2024-01-08,2.08,2.10,2.07,2.08,2.08,8755000,0.000000,0.000000,0.000000,0.011285,0.010757,1.049084,51.1977,7277.KL
5,2024-01-09,2.07,2.08,2.06,2.07,2.07,6147500,-0.009999,0.000000,0.009999,0.010478,0.010702,0.979069,49.4711,7277.KL
6,2024-01-10,2.07,2.07,2.04,2.07,2.07,9891400,0.000000,0.000000,0.000000,0.009729,0.009937,0.979068,49.4711,7277.KL
7,2024-01-11,2.06,2.06,1.99,2.01,2.01,7168100,-0.059999,0.000000,0.059999,0.009034,0.013512,0.668590,40.0691,7277.KL
8,2024-01-12,2.02,2.05,2.01,2.05,2.05,9055700,0.040000,0.040000,0.000000,0.011245,0.012546,0.896301,47.2657,7277.KL
9,2024-01-15,2.05,2.05,1.97,1.98,1.98,10595300,-0.069999,0.000000,0.069999,0.010441,0.016649,0.627124,38.5418,7277.KL


In [84]:
### Create a blank dataframe
final_stocks_history = pd.DataFrame()

### Then apply the formula for all available stocks
for stock_i in dim_stock["Code"]:
    try:
        single_run = stock_history_function(stock_i)
        final_stocks_history = pd.concat([final_stocks_history, single_run], ignore_index=True)
    except Exception as e:
        continue

### Print the final output
print(final_stocks_history.head())    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['6436.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')


C:\Users\Ryan1\AppData\Local\Temp\ipykernel_22204\2727379142.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_stocks_history = pd.concat([final_stocks_history, single_run], ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['5270.KL']: Exception('%ticker%: No timezone found, symbol may be delisted')


...

C:\Users\Ryan1\AppData\Local\Temp\ipykernel_22204\2727379142.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_stocks_history = pd.concat([final_stocks_history, single_run], ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['0256.KL']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


C:\Users\Ryan1\AppData\Local\Temp\ipykernel_22204\2727379142.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_stocks_history = pd.concat([final_stocks_history, single_run], ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### 5. Export Data to Excel
This section is to export the run Python output to an Excel File. This step will replace the old file with new file every time it runs.

In [85]:

# Set the batch size
batch_size = 500000 #500k rows per file

# Calculate the number of batches needed
num_batches = round((len(final_stocks_history) + batch_size - 1) // batch_size)

# Export each batch to a separate Excel file
for i in range(num_batches):
    start_idx = i * batch_size 
    end_idx = min((i + 1) * batch_size, len(final_stocks_history))

    # Extract the current batch
    current_batch = final_stocks_history.iloc[start_idx:end_idx]

    # Specify the Excel file path for the current batch
    excel_file_path = f'output/batch_{i + 1}.xlsx'

    # Export the current batch to Excel
    current_batch.to_excel(excel_file_path, index=False)

    print(f"Batch {i + 1} exported to {excel_file_path}")

Batch 1 exported to output/batch_1.xlsx
